In [86]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier


In [96]:
df = pd.read_csv('_gamers.csv')
df

,login,gender,age,About Me,Interests,City,Age Low,Age High,Alan Wake,Apex Legends,...,google_user_433,google_user_4365,google_user_4487,google_user_4673,google_user_4692,hayaati77,hewie,mitsu,rb201513,theFUM3
0,RoDjo,0,27,Тихо и мирно предпринимаю попытки (преимуществ...,"История, волонтерство, география, военная техн...",0,16,40,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Krator1,0,26,"Обычный парень, чуть-чуть хиккан,большую часть...","Игры, аниме, музыка, сирики, стримы, да и так,...",0,16,81,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Baka_Sobacka,0,22,Одинокий романтик. В комнате полной людей я ча...,"Меня всегда интересовало фэнтези и магия, обож...",0,20,30,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Electronik,0,31,0,0,0,20,40,0,0,...,0,0,0,0,0,0,0,0,0,0
4,google_user_3696,0,36,0,0,0,18,40,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,Sh0ckeR44,0,37,"Геймер, ретро геймер, стример, блогер, поэт - ...","Огромная вселенная игровой индустрии, также ст...",Саратов,17,81,0,0,...,0,0,0,0,0,0,0,0,0,0
125,KanZaki,1,27,0,0,Тверь,20,35,0,0,...,0,0,0,0,0,0,0,0,0,0
126,Cogneter,0,33,"Создал этот сайт, пытаюсь его развивать. :)","Программирование, геймдизайн, технологии, наук...",Москва,18,30,0,0,...,0,0,0,0,0,0,0,0,0,0
127,Literally,1,26,0,0,Москва,20,35,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Список стоп-слов для русского языка
russian_stop_words = [
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так',
    'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было',
    'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг',
    'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж',
    'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть',
    'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего',
    'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого',
    'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас',
    'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть',
    'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много',
    'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше',
    'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'
]

# Создание векторизаторов для каждого текстового столбца
tfidf_vectorizer_about_me = TfidfVectorizer(stop_words=russian_stop_words)
tfidf_vectorizer_interests = TfidfVectorizer(stop_words=russian_stop_words)
tfidf_vectorizer_city = TfidfVectorizer(stop_words=russian_stop_words)

# Преобразование текста в числовые признаки с помощью TF-IDF
tfidf_matrix_about_me = tfidf_vectorizer_about_me.fit_transform(df['About Me'].astype(str))
tfidf_matrix_interests = tfidf_vectorizer_interests.fit_transform(df['Interests'].astype(str))
tfidf_matrix_city = tfidf_vectorizer_city.fit_transform(df['City'].astype(str))

# Преобразование TF-IDF матриц в DataFrame
df_tfidf_about_me = pd.DataFrame(tfidf_matrix_about_me.toarray(), columns=tfidf_vectorizer_about_me.get_feature_names_out())
df_tfidf_interests = pd.DataFrame(tfidf_matrix_interests.toarray(), columns=tfidf_vectorizer_interests.get_feature_names_out())
df_tfidf_city = pd.DataFrame(tfidf_matrix_city.toarray(), columns=tfidf_vectorizer_city.get_feature_names_out())

# Объединение TF-IDF матриц с исходным DataFrame
df = pd.concat([df, df_tfidf_about_me, df_tfidf_interests, df_tfidf_city], axis=1)

# Удаление исходных текстовых колонок
df.drop(columns=['About Me', 'Interests', 'City', 'login'], inplace=True)
df

,gender,age,Age Low,Age High,Alan Wake,Apex Legends,ArcheAge,Bioshock Infinite,Borderlands,Borderlands 2,...,ташкент,тверь,темиртау,тольятти,тюмень,украина,уральск,челны,челябинск,черкассы
0,0,27,16,40,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,26,16,81,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,22,20,30,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,31,20,40,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,36,18,40,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0,37,17,81,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,1,27,20,35,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,0,33,18,30,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127,1,26,20,35,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
df_copy = df.copy()

In [89]:
print(df.columns.tolist())

['gender', 'age', 'Age Low', 'Age High', 'Alan Wake', 'Apex Legends', 'ArcheAge', 'Bioshock Infinite', 'Borderlands', 'Borderlands 2', 'Borderlands 3', 'Call of Duty', 'Company of Heroes 2', 'Counter-Strike', 'Cyberpunk 2077', 'Dark Souls', 'Dead Island', 'Dead by Daylight', 'Detroit: Become Human', 'Deus Ex', 'Diablo', 'Diablo II', 'Diablo III', 'Disco Elysium', 'Dishonored', 'Divinity: Original Sin II', 'Dota 2', 'Endless Space 2', 'Fable', 'Fallout', 'Fallout: New Vegas', 'Far Cry', 'Fortnite', 'Genshin Impact', 'Grand Theft Auto V', 'Grand Theft Auto: San Andreas', 'Grand Theft Auto: Vice City', 'Half-Life', 'Half-Life 2', 'Hearthstone', 'Heroes of Might and Magic III: The Restoration of Erathia', 'Heroes of Might and Magic IV', 'Heroes of Might and Magic V', 'Killing Floor 2', 'Kingdom Come: Deliverance', 'League of Legends', 'League of Legends: Wild Rift', 'Left 4 Dead', 'Left 4 Dead 2', 'Life is Strange', 'Mafia', 'Mass Effect', 'Mass Effect 2', 'Metro 2033', 'Metro Exodus', 'Mi

In [90]:
# Список всех целевых переменных
target_columns = ['AjdosGulev', 'Altmer', 'Akigis', 'AnastasijaSHustova', 'AndrejOkunev', 'AleksejGejmer', 'Alica_Melica',
                    'AndrejMaksimov', 'Anko', 'AnnaVajt', 'Apofitus', 'Asker', 'Baka_Sobacka', 'CHYHKA', 'CHulpanSHajdullina',
                    'Cogneter', 'Combobreaker', 'DanilAstapenko', 'Darkness', 'DemoNova', 'Denis12345667', 'DenisFeofanov',
                    'DenisJUrnero', 'DenisMalyshev', 'Dima27_', 'DmitrijZenkin', 'EHduardObolenskij', 'EgorHabarov', 'Electronik',
                    'ElvenNeko', 'Enclave', 'Evegard', 'Friends', 'GenaVasiliev', 'Gfg', 'GhostHelios', 'Gopit', 'Granix', 'HSTT',
                    'Halcon', 'HanteRina', 'IljaCHivkin', 'JUnusFajzaliev', 'JackLipat', 'Kaita', 'KanZaki', 'KirillLuckevich',
                    'Kiskamav', 'Kley', 'Kohas', 'Kotoponchik', 'Krator1', 'Kreyvus', 'KsenijaLebedeva', 'Literally', 'LizMajers',
                    'Lorindol', 'MadGamer', 'Meinumo', 'Meowmeow', 'MrOkulus', 'NastjaIvanova', 'NikitaDunaevskij', 'NikitaGorit',
                    'ObiWan', 'OlegSkorobogatov', 'Paff', 'Pop', 'Pranik', 'RainbowFox', 'Razvedosa', 'Rexolik', 'RoDjo', 'RobotUnicorn',
                    'RomanSaraev', 'Sapphire362', 'Sh0ckeR44', 'Shax', 'Simmetric', 'Tallesa', 'Ter_ded', 'Tk095', 'UED', 'ValerijRepchenko',
                    'Veteran', 'VladHandler', 'VladimirDubrovin', 'VladislavMartin', 'WatermelonSeal', 'XXX69', 'YonakaHitsugaya', 'Your_Sister',
                    'Yuji', '_Devil_', 'bianconeri', 'c4', 'empty', 'fr0st2o11', 'gawric', 'goodman_dd', 'google_user_1768', 'google_user_1797',
                    'google_user_2178', 'google_user_2253', 'google_user_2386', 'google_user_2547', 'google_user_2591', 'google_user_2716',
                    'google_user_2732', 'google_user_2853', 'google_user_2854', 'google_user_2860', 'google_user_3319', 'google_user_3383',
                    'google_user_3388', 'google_user_3429', 'google_user_3583', 'google_user_3616', 'google_user_3696', 'google_user_3759',
                    'google_user_3856', 'google_user_433', 'google_user_4365', 'google_user_4487', 'google_user_4673', 'google_user_4692',
                    'hayaati77', 'hewie', 'mitsu', 'rb201513', 'theFUM3']


threshold = 0.2

# Создаем новый DataFrame для хранения только важных признаков
new_df = pd.DataFrame()

# Проходим по каждой целевой переменной
for target_variable in target_columns:
    # Определяем входные данные и целевую переменную для текущей целевой переменной
    X = df.drop(columns=target_variable)
    y = df[target_variable]

    # Инициализируем модель случайного леса
    rf_model = RandomForestClassifier()

    # Обучаем модель
    rf_model.fit(X, y)

    # Получаем важности признаков
    feature_importances = rf_model.feature_importances_

    # Определяем признаки с важностью выше порога
    important_indices = np.where(feature_importances > threshold)[0]
    important_features = X.columns[important_indices]

    # Добавляем важные признаки к новому DataFrame
    for col in important_features:
        new_df[col] = df[col]

    # Добавляем целевую переменную к новому DataFrame
    new_df[target_variable] = y

new_df


<ipython-input-90-7a703dfa0ad9>:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[target_variable] = y
<ipython-input-90-7a703dfa0ad9>:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[target_variable] = y
<ipython-input-90-7a703dfa0ad9>:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
 

,AjdosGulev,Altmer,Akigis,AnastasijaSHustova,AndrejOkunev,AleksejGejmer,Alica_Melica,AndrejMaksimov,Anko,AnnaVajt,...,google_user_433,google_user_4365,google_user_4487,google_user_4673,google_user_4692,hayaati77,hewie,mitsu,rb201513,theFUM3
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
threshold = 0.5

# Вычисление матрицы корреляции
corr_matrix = df_copy.drop(columns=target_columns).corr().abs()

# Выбор верхнего треугольника матрицы корреляции
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Нахождение индексов столбцов с корреляцией больше заданного порога
to_drop_corr = [column for column in upper.columns if any(upper[column] > threshold)]

# Удаление лишних столбцов
df_copy.drop(columns=to_drop_corr, inplace=True)

df_copy

,gender,age,Age Low,Age High,Alan Wake,Apex Legends,ArcheAge,Bioshock Infinite,Borderlands,Borderlands 2,...,малаховка,минск,москва,набережные,оскол,петербург,рефтинский,тверь,тольятти,уральск
0,0,27,16,40,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,26,16,81,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,22,20,30,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,31,20,40,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,36,18,40,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0,37,17,81,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,1,27,20,35,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
126,0,33,18,30,0,0,0,0,1,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127,1,26,20,35,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Метод случайного леса (Random Forest):

In [107]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Определение признаков и целевых переменных
X = df_copy.drop(columns=target_columns)  # Признаки
y = df_copy[target_columns]  # Целевые переменные

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Инициализация и обучение модели Random Forest
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# Предсказание на тестовом наборе данных
y_pred = rf_classifier.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Отчет о классификации
print(classification_report(y_test, y_pred))


Accuracy: 0.0
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         0
          17 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [113]:
y

,AjdosGulev,Altmer,Akigis,AnastasijaSHustova,AndrejOkunev,AleksejGejmer,Alica_Melica,AndrejMaksimov,Anko,AnnaVajt,...,google_user_433,google_user_4365,google_user_4487,google_user_4673,google_user_4692,hayaati77,hewie,mitsu,rb201513,theFUM3
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
X = df_copy.drop(columns=target_columns)  # Признаки
y = df_copy[target_columns]  # Целевые переменные

# Предположим, что X содержит ваши признаки, а y содержит целевую переменную (дружат или нет)
# X должен содержать только признаки, без целевой переменной

# Инициализация kNN модели с числом соседей равным 4 (топ 3 похожих плюс сам пользователь)
knn_model = NearestNeighbors(n_neighbors=4)

# Обучение модели
knn_model.fit(X)

# Получение индексов и расстояний до ближайших соседей для каждого пользователя
distances, indices = knn_model.kneighbors(X)

# Для каждого пользователя
for i in range(len(indices)):
    print(f"Для пользователя {i}:")

    # Получение индексов топ 3 похожих пользователей
    top3_similar_users = indices[i][1:]  # исключаем самого пользователя

    # Проверяем, дружат ли топ 3 похожих пользователей с текущим пользователем
    for user_index in top3_similar_users:
      print(user_index)
      if y.iloc[i, user_index] == 1:  # если значение равно 1
            print(f"Пользователь {i} дружит с пользователем {user_index}")
      else:
            print(f"Пользователь {i} НЕ дружит с пользователем {user_index}")

Для пользователя 0:
6
Пользователь 0 НЕ дружит с пользователем 6
88
Пользователь 0 НЕ дружит с пользователем 88
76
Пользователь 0 НЕ дружит с пользователем 76
Для пользователя 1:
78
Пользователь 1 НЕ дружит с пользователем 78
62
Пользователь 1 НЕ дружит с пользователем 62
103
Пользователь 1 НЕ дружит с пользователем 103
Для пользователя 2:
5
Пользователь 2 НЕ дружит с пользователем 5
48
Пользователь 2 НЕ дружит с пользователем 48
119
Пользователь 2 НЕ дружит с пользователем 119
Для пользователя 3:
110
Пользователь 3 НЕ дружит с пользователем 110
37
Пользователь 3 НЕ дружит с пользователем 37
88
Пользователь 3 НЕ дружит с пользователем 88
Для пользователя 4:
15
Пользователь 4 НЕ дружит с пользователем 15
36
Пользователь 4 НЕ дружит с пользователем 36
37
Пользователь 4 НЕ дружит с пользователем 37
Для пользователя 5:
117
Пользователь 5 НЕ дружит с пользователем 117
2
Пользователь 5 НЕ дружит с пользователем 2
97
Пользователь 5 НЕ дружит с пользователем 97
Для пользователя 6:
88
Пользоват